In [1]:
import org.apache.spark.sql.SparkSession

Intitializing Scala interpreter ...

Spark Web UI available at http://MSI:4040
SparkContext available as 'sc' (version = 3.0.0, master = local[*], app id = local-1605139170667)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession


In [2]:
val spark = SparkSession.builder().getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@6b8c244b


# DataFrames Overview

In [3]:
val df = spark.read.option("header","true").option("inferSchema","true").csv("../../data/ml_scala/CitiGroup2006_2008")

df: org.apache.spark.sql.DataFrame = [Date: string, Open: double ... 4 more fields]


In [4]:
df.head(5)

res0: Array[org.apache.spark.sql.Row] = Array([2006-01-03,490.0,493.8,481.1,492.9,1537660], [2006-01-04,488.6,491.0,483.5,483.8,1871020], [2006-01-05,484.4,487.8,484.0,486.2,1143160], [2006-01-06,488.8,489.0,482.0,486.2,1370250], [2006-01-09,486.0,487.4,483.0,483.9,1680740])


In [5]:
for(row <- df.head(5)){
    println(row)
}

[2006-01-03,490.0,493.8,481.1,492.9,1537660]
[2006-01-04,488.6,491.0,483.5,483.8,1871020]
[2006-01-05,484.4,487.8,484.0,486.2,1143160]
[2006-01-06,488.8,489.0,482.0,486.2,1370250]
[2006-01-09,486.0,487.4,483.0,483.9,1680740]


In [6]:
df.columns

res2: Array[String] = Array(Date, Open, High, Low, Close, Volume)


In [7]:
df.describe().show()

+-------+----------+------------------+-----------------+------------------+------------------+-----------------+
|summary|      Date|              Open|             High|               Low|             Close|           Volume|
+-------+----------+------------------+-----------------+------------------+------------------+-----------------+
|  count|       755|               755|              755|               755|               755|              755|
|   mean|      null| 386.0923178807949|390.6590596026489|380.80170860927143| 385.3421456953643|6308596.382781457|
| stddev|      null|149.32301134820133|148.5151130063523|150.53136890891344|149.83310074439177| 8099892.56297633|
|    min|2006-01-03|              54.4|             55.3|              30.5|              37.7|           632860|
|    max|2008-12-31|             566.0|            570.0|             555.5|             564.1|        102869289|
+-------+----------+------------------+-----------------+------------------+------------

In [8]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)



In [9]:
// Single Column
df.select("Volume").show(5)

+-------+
| Volume|
+-------+
|1537660|
|1871020|
|1143160|
|1370250|
|1680740|
+-------+
only showing top 5 rows



In [10]:
// Multiple Columns
df.select($"Date",$"Close").show(5)

+----------+-----+
|      Date|Close|
+----------+-----+
|2006-01-03|492.9|
|2006-01-04|483.8|
|2006-01-05|486.2|
|2006-01-06|486.2|
|2006-01-09|483.9|
+----------+-----+
only showing top 5 rows



In [11]:
// Create New Columns
val df2 = df.withColumn("HighPlusLow",df("High") + df("Low"))
df2.show(5)
df2.printSchema()

+----------+-----+-----+-----+-----+-------+-----------------+
|      Date| Open| High|  Low|Close| Volume|      HighPlusLow|
+----------+-----+-----+-----+-----+-------+-----------------+
|2006-01-03|490.0|493.8|481.1|492.9|1537660|974.9000000000001|
|2006-01-04|488.6|491.0|483.5|483.8|1871020|            974.5|
|2006-01-05|484.4|487.8|484.0|486.2|1143160|            971.8|
|2006-01-06|488.8|489.0|482.0|486.2|1370250|            971.0|
|2006-01-09|486.0|487.4|483.0|483.9|1680740|            970.4|
+----------+-----+-----+-----+-----+-------+-----------------+
only showing top 5 rows

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- HighPlusLow: double (nullable = true)



df2: org.apache.spark.sql.DataFrame = [Date: string, Open: double ... 5 more fields]


In [12]:
// Rename Column
df2.select(df2("HighPlusLow").as("HPL")).show(5)

+-----------------+
|              HPL|
+-----------------+
|974.9000000000001|
|            974.5|
|            971.8|
|            971.0|
|            970.4|
+-----------------+
only showing top 5 rows



# Operations

In [13]:
import spark.implicits._  // Allows use of $ sign SCALA notation

import spark.implicits._


In [14]:
// Grab all rows where column meets single condition - SCALA notation
df.filter($"Close" > 480).show(5)

+----------+-----+-----+-----+-----+-------+
|      Date| Open| High|  Low|Close| Volume|
+----------+-----+-----+-----+-----+-------+
|2006-01-03|490.0|493.8|481.1|492.9|1537660|
|2006-01-04|488.6|491.0|483.5|483.8|1871020|
|2006-01-05|484.4|487.8|484.0|486.2|1143160|
|2006-01-06|488.8|489.0|482.0|486.2|1370250|
|2006-01-09|486.0|487.4|483.0|483.9|1680740|
+----------+-----+-----+-----+-----+-------+
only showing top 5 rows



In [15]:
// Grab all rows where column meets single condition - SQL notation
df.filter("Close > 480").show(5)

+----------+-----+-----+-----+-----+-------+
|      Date| Open| High|  Low|Close| Volume|
+----------+-----+-----+-----+-----+-------+
|2006-01-03|490.0|493.8|481.1|492.9|1537660|
|2006-01-04|488.6|491.0|483.5|483.8|1871020|
|2006-01-05|484.4|487.8|484.0|486.2|1143160|
|2006-01-06|488.8|489.0|482.0|486.2|1370250|
|2006-01-09|486.0|487.4|483.0|483.9|1680740|
+----------+-----+-----+-----+-----+-------+
only showing top 5 rows



In [16]:
// Filter on multiple conditions - SCALA notation
df.filter($"Close" < 480 && $"High" < 480).show(5)

+----------+-----+-----+-----+-----+-------+
|      Date| Open| High|  Low|Close| Volume|
+----------+-----+-----+-----+-----+-------+
|2006-01-20|472.1|474.0|456.3|456.9|4781930|
|2006-01-23|460.0|463.8|457.0|460.0|2025500|
|2006-01-24|462.9|463.6|459.9|460.1|2083740|
|2006-01-25|461.4|463.7|460.1|462.3|1591940|
|2006-01-26|465.5|475.5|464.5|470.1|1988600|
+----------+-----+-----+-----+-----+-------+
only showing top 5 rows



In [17]:
// Filter on multiple conditions - SQL notation
df.filter("Close < 480 AND High < 480").show(5)

+----------+-----+-----+-----+-----+-------+
|      Date| Open| High|  Low|Close| Volume|
+----------+-----+-----+-----+-----+-------+
|2006-01-20|472.1|474.0|456.3|456.9|4781930|
|2006-01-23|460.0|463.8|457.0|460.0|2025500|
|2006-01-24|462.9|463.6|459.9|460.1|2083740|
|2006-01-25|461.4|463.7|460.1|462.3|1591940|
|2006-01-26|465.5|475.5|464.5|470.1|1988600|
+----------+-----+-----+-----+-----+-------+
only showing top 5 rows



In [18]:
// Assign to new dataframe
val CH_low = df.filter($"Close" < 480 && $"High" < 480)
CH_low.count()

CH_low: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Date: string, Open: double ... 4 more fields]
res13: Long = 397


In [19]:
// Filter on equality (Have to use three '=' signs)
df.filter($"High"=== 484.40).show()

+----------+-----+-----+-----+-----+-------+
|      Date| Open| High|  Low|Close| Volume|
+----------+-----+-----+-----+-----+-------+
|2006-04-27|472.0|484.4|471.5|481.5|2464800|
+----------+-----+-----+-----+-----+-------+



In [20]:
// Correlation between 2 columns
df.select(corr("High","Low")).show()

+------------------+
|   corr(High, Low)|
+------------------+
|0.9992999172726325|
+------------------+



In [21]:
// Other Operations and Useful Functions
// http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$

# GroupBy and Aggregate Functions

In [22]:
val df = spark.read.option("header","true").option("inferSchema","true").csv("../../data/ml_scala/Sales.csv")
df.show()
df.printSchema()
df.describe().show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|  200|
|   GOOG|Charlie|  120|
|   GOOG|  Frank|  340|
|   MSFT|   Tina|  600|
|   MSFT|    Amy|  124|
|   MSFT|Vanessa|  243|
|     FB|   Carl|  870|
|     FB|  Sarah|  350|
+-------+-------+-----+

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: integer (nullable = true)

+-------+-------+-------+------------------+
|summary|Company| Person|             Sales|
+-------+-------+-------+------------------+
|  count|      8|      8|                 8|
|   mean|   null|   null|           355.875|
| stddev|   null|   null|259.29819430807567|
|    min|     FB|    Amy|               120|
|    max|   MSFT|Vanessa|               870|
+-------+-------+-------+------------------+



df: org.apache.spark.sql.DataFrame = [Company: string, Person: string ... 1 more field]


In [23]:
// Group by One Column
df.groupBy("Company").mean().show()
df.groupBy("Company").count().show()
df.groupBy("Company").max().show()
df.groupBy("Company").min().show()
df.groupBy("Company").sum().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+

+-------+-----+
|Company|count|
+-------+-----+
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   GOOG|       340|
|     FB|       870|
|   MSFT|       600|
+-------+----------+

+-------+----------+
|Company|min(Sales)|
+-------+----------+
|   GOOG|       120|
|     FB|       350|
|   MSFT|       124|
+-------+----------+

+-------+----------+
|Company|sum(Sales)|
+-------+----------+
|   GOOG|       660|
|     FB|      1220|
|   MSFT|       967|
+-------+----------+



In [24]:
// Other Aggregate Functions
// http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$
df.select(countDistinct("Sales")).show() //approxCountDistinct
df.select(sumDistinct("Sales")).show()
df.select(variance("Sales")).show()
df.select(stddev("Sales")).show() //avg,max,min,sum,stddev
df.select(collect_set("Sales")).show()
df.select(sum("Sales")).show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                    8|
+---------------------+

+-------------------+
|sum(DISTINCT Sales)|
+-------------------+
|               2847|
+-------------------+

+-----------------+
|  var_samp(Sales)|
+-----------------+
|67235.55357142855|
+-----------------+

+------------------+
|stddev_samp(Sales)|
+------------------+
|259.29819430807567|
+------------------+

+--------------------+
|  collect_set(Sales)|
+--------------------+
|[350, 340, 870, 1...|
+--------------------+

+----------+
|sum(Sales)|
+----------+
|      2847|
+----------+



In [25]:
// Order by with Desc order
df.orderBy($"Sales".desc).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|  870|
|   MSFT|   Tina|  600|
|     FB|  Sarah|  350|
|   GOOG|  Frank|  340|
|   MSFT|Vanessa|  243|
|   GOOG|    Sam|  200|
|   MSFT|    Amy|  124|
|   GOOG|Charlie|  120|
+-------+-------+-----+



# Missing Data

In [26]:
val df = spark.read.option("header","true").option("inferSchema","true").csv("../../data/ml_scala/ContainsNull.csv")
df.show()
df.printSchema()
df.describe().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)

+-------+----+-----+-----------------+
|summary|  Id| Name|            Sales|
+-------+----+-----+-----------------+
|  count|   4|    2|                2|
|   mean|null| null|            400.5|
| stddev|null| null|78.48885271170677|
|    min|emp1|Cindy|            345.0|
|    max|emp4| John|            456.0|
+-------+----+-----+-----------------+



df: org.apache.spark.sql.DataFrame = [Id: string, Name: string ... 1 more field]


In [27]:
// Drop any row with na
df.na.drop.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [28]:
// Drop with a threshold on number of NA in a row.  Drop any row that has less than a minimum number of non-null values
df.na.drop(2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [29]:
// Fill in null-values
df.na.fill(100).show()   // Note how it auto detects column based on datatype
df.na.fill("Nick").show()  // This would work on all columns with datatype specified in the fill statement

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|100.0|
|emp2| null|100.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| Nick| null|
|emp3| Nick|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [30]:
// Specify which column to handle NA values
df.na.fill("New Name",Array("Name")).show()
df.na.fill(200, Array("Sales")).show()

+----+--------+-----+
|  Id|    Name|Sales|
+----+--------+-----+
|emp1|    John| null|
|emp2|New Name| null|
|emp3|New Name|345.0|
|emp4|   Cindy|456.0|
+----+--------+-----+

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|200.0|
|emp2| null|200.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [31]:
// Sample imputation using mean of Sales to fill in the missings sales values.
df.describe().show()
df.na.fill(400.5, Array("Sales")).show()

+-------+----+-----+-----------------+
|summary|  Id| Name|            Sales|
+-------+----+-----+-----------------+
|  count|   4|    2|                2|
|   mean|null| null|            400.5|
| stddev|null| null|78.48885271170677|
|    min|emp1|Cindy|            345.0|
|    max|emp4| John|            456.0|
+-------+----+-----+-----------------+

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [32]:
// Include both steps
val df2 = df.na.fill(400.5, Array("Sales"))
df2.na.fill("Missing Name", Array("Name")).show()

+----+------------+-----+
|  Id|        Name|Sales|
+----+------------+-----+
|emp1|        John|400.5|
|emp2|Missing Name|400.5|
|emp3|Missing Name|345.0|
|emp4|       Cindy|456.0|
+----+------------+-----+



df2: org.apache.spark.sql.DataFrame = [Id: string, Name: string ... 1 more field]


# Date and Timestamps

In [33]:
val df = spark.read.option("header","true").option("inferSchema","true").csv("../../data/ml_scala/CitiGroup2006_2008")
df.printSchema()
df.show(5)

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)

+----------+-----+-----+-----+-----+-------+
|      Date| Open| High|  Low|Close| Volume|
+----------+-----+-----+-----+-----+-------+
|2006-01-03|490.0|493.8|481.1|492.9|1537660|
|2006-01-04|488.6|491.0|483.5|483.8|1871020|
|2006-01-05|484.4|487.8|484.0|486.2|1143160|
|2006-01-06|488.8|489.0|482.0|486.2|1370250|
|2006-01-09|486.0|487.4|483.0|483.9|1680740|
+----------+-----+-----+-----+-----+-------+
only showing top 5 rows



df: org.apache.spark.sql.DataFrame = [Date: string, Open: double ... 4 more fields]


In [34]:
// Lot's of options here
// http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$@add_months(startDate:org.apache.spark.sql.Column,numMonths:Int):org.apache.spark.sql.Column

In [35]:
df.select(month(df("Date"))).show(5)

+-----------+
|month(Date)|
+-----------+
|          1|
|          1|
|          1|
|          1|
|          1|
+-----------+
only showing top 5 rows



In [36]:
df.select(year(df("Date"))).show(5)

+----------+
|year(Date)|
+----------+
|      2006|
|      2006|
|      2006|
|      2006|
|      2006|
+----------+
only showing top 5 rows



In [37]:
val df2 = df.withColumn("Year", year(df("Date")))
val dfavg = df2.groupBy("Year").mean()

dfavg.select($"Year",$"avg(Close)").show()

+----+------------------+
|Year|        avg(Close)|
+----+------------------+
|2007| 477.8203984063745|
|2006| 489.2697211155379|
|2008|190.48893280632404|
+----+------------------+



df2: org.apache.spark.sql.DataFrame = [Date: string, Open: double ... 5 more fields]
dfavg: org.apache.spark.sql.DataFrame = [Year: int, avg(Open): double ... 5 more fields]
